[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lorenzobasile/DeepLearning2022/blob/main/3_regularization.ipynb)

# Lab 3

In [ ]:
import torch
import matplotlib.pyplot as plt

### Recap from previous Labs

* We saw how to handle `torch` tensors and how to use them for our computational purposes;
* We defined our first MLP models both for regression and classification;
* We now know how to define loss functions, differentiate them and use their gradients to train a simple neural network;
* We saw our first (simple) example of a real application of DL: MNIST.

### Today

We will introduce the idea of **regularization** and the main techniques employed to regularize neural network learning and to prevent overfitting and we will see the main techniques used to **normalize** the data we feed into a neural network.


# Regularization

## What is regularization? Why do we need it?

Regularization is the name of a very broad and diverse set of techniques that are used to improve the way neural networks (but, more generally, any ML model) learn. As we will see, not all techniques are based on the same working principle: some insert some sort of bias in the solution that we are looking for, while others directly impact the architecture of the model or the length of the training procedure.

What all these techniques have in common is the reason why they are implemented: to avoid or contain *overfitting*.

Overfitting is a widespread phenomenon in Machine Learning. It happens when a model captures *too well* the variability of training data, meaning that the model is fitting the noise and irrelevant features of training data, becoming unable to generalize on unseen data (i.e. test data).

To have an idea of what overfitting is we can think of a simple regression task:

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/points.png?raw=1" width="500"/>

Since we have 10 points there exists a unique polynomial of degree 9 that passes through these points, perfectly fitting (overfitting) our data:

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/poly.png?raw=1" width="500"/>

Paying a small price in terms of MSE, we could obtain a decent fit with a linear model (polynomial of degree 1):

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/lin.png?raw=1" width="500"/>

It is obvious not to decide which model works best: in principle nothing is telling us that we shouldn't fit a polynomial of degree 9 for this task. However, in general we strive for simpler models, as they are clearer, more explainable and better at capturing general patterns in the data, which usually makes them better at generalizing on unseen data points.



Please refer to [chapter 3](http://neuralnetworksanddeeplearning.com/chap3.html) of Michael Nielsen's book for a more in-depth analysis of this example and of some of the techniques we will see in this lab.

## A set of tools for regularization

Regularization in DL comes in the form of different tools. We can have:

1. Penalty terms in loss functions (e.g. L1 and L2 norm regularization) which introduce bias in our parameters by actively reducing the magnitude of some weights:
    * L2 norm regularization is also called Ridge regularization or **weight decay**
    * L1 norm regularization is also called LASSO regularization
    * they were originally implemented in linear regression models as a way to infuse *inductive bias* in models originally thought to rely on the complete unbiasedness on training data
2. Dropout, a technique [patented by Google](https://patents.google.com/patent/US9406017B2/en) which consists in randomly *dropping* some neurons from a given layer to prevent overfitting.
3. Early stopping

## The XOR problem



We will be dealing with a simple toy problem that is very well known in the Deep Learning community: the XOR problem.

It is a binary classification problem in which the input is 2-dimensional and the output is 1 if the two signs coincide and 0 otherwise. The problem is trivial but it draws importance from the fact that it is one of the simplest classification tasks that cannot be solved by using a linear model.

We add some noise in the data by randomly switching the label for 5% of the data.

In [ ]:
torch.manual_seed(0)
X=torch.randn((1000,2))
Y=torch.logical_or(torch.all(X.ge(0), dim=1), torch.all(X.le(0), dim=1)).reshape(-1,1)

p=int(0.05*Y.shape[0])
perm = torch.randperm(Y.shape[0])
Y[perm[:p]]=~Y[perm[:p]]
Y=Y.float()


We can visualize the data belonging to the two classes: it is clearly visible that it is impossible to separate the classes with a linear boundary.

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(X[:,0], X[:,1], s=10, c=Y)

We can use a PyTorch built-in function to randomly split the dataset into a training and a test set.

In [ ]:
dataset=torch.utils.data.TensorDataset(X, Y)
trainset, testset=torch.utils.data.random_split(dataset, [500, 500])

trainloader=torch.utils.data.DataLoader(trainset, batch_size=16, shuffle=True)
testloader=torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False)

And we define our binary classifier as an MLP with three layers, ReLU activations and a final Sigmoid activation. We will be using the `BCELoss` (the binary version of Cross Entropy): please note that this loss function in torch requires the output to be 1D and normalized, i.e. **the Sigmoid is needed** (recall that Softmax is not needed when doing multi-class classification).

In [ ]:
class Classifier(torch.nn.Module):
    def __init__(self, input_dim=2, linear=False):
        super().__init__()
        self.layer1 = torch.nn.Linear(in_features=input_dim, out_features=4, bias=True)
        self.layer2 = torch.nn.Linear(in_features=4, out_features=4, bias=True)
        self.layer3 = torch.nn.Linear(in_features=4, out_features=1, bias=True)
        if linear:
            self.activation = torch.nn.Identity()
        else:
            self.activation = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x=self.activation(self.layer1(x))
        x=self.activation(self.layer2(x))
        x=self.sigmoid(self.layer3(x))
        return x

## Weight decay



The first kind of regularization we will deal with is Ridge regularization (or L2 regularization, or simply Weight Decay).

Weight Decay  is a simple technique which *appends* a penalty term to the loss function equation. The term is based upon the $l_2$ norm of the weights.

Given our original loss function $L (\hat{y}, y)$ and our parameter vector $\Theta$, our new loss will be:

$$
L (\hat{y}, y) + \lambda \cdot \vert\vert \Theta \vert\vert_2^2
$$

the parameter $\lambda$ (also called weight decay) controls the strength of the regularization. $\lambda$ too high means that the model will not concentrate well enough on the original objective ($L$), hence it will not perform well.

In torch, instead of inserting our penalty term in the loss function, we specify the weight decay parameter in our optimizer.

In [ ]:
l2_model=Classifier()
optimizer=torch.optim.Adam(l2_model.parameters(), lr=1e-2, weight_decay=0.01)
loss=torch.nn.BCELoss()

In [ ]:
def get_accuracy(model, dataloader):
    model.eval()
    with torch.no_grad():
        correct=0
        for x, y in iter(dataloader):
            out=model(x)
            correct+=(torch.round(out)==y).sum()
        return correct/len(dataloader.dataset)
def get_l2_norm(model):
    total_norm=0
    for p in model.parameters():
        param_norm = torch.norm(p.data, p=2)
        total_norm += param_norm.item() ** 2
    total_norm = total_norm ** (1. / 2)
    return total_norm


We can quickly obtain good performance on our task while limiting the weight norm increase.

In [ ]:
def train(model, optimizer, trainloader, testloader):
    epochs=10
    for epoch in range(epochs):
        print("Test accuracy: ", get_accuracy(model, testloader))
        model.train()    
        print("Epoch: ", epoch)
        for x, y in iter(trainloader):
            out=model(x)
            l=loss(out, y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
    print("Final accuracy: ", get_accuracy(model, testloader))


In [ ]:
train(l2_model, optimizer, trainloader, testloader)
print("Final norm: ", get_l2_norm(l2_model))

What would happen if we replace the `ReLU` with `Identity`?

In [ ]:
linear_model=Classifier(linear=True)
optimizer=torch.optim.Adam(linear_model.parameters(), lr=1e-2, weight_decay=0.01)
train(linear_model, optimizer, trainloader, testloader)

## LASSO


LASSO (or L1 norm) regularization is analogous to weight decay. The equation is:

$$
L (\hat{y}, y) + \lambda \cdot \vert\vert \Theta \vert\vert_1
$$

where $\vert\vert x \vert\vert_1 = \sum_{j=1}^d \vert x_j \vert$

unlike weight decay, torch does not provide a built-in for L1 regularization. To implement it you would have to customize the optimizer, create a custom loss function or simply tweak the training loop as we will see.

L1 regularization is often used as a way to enforce *sparsity* in the model weights: in fact, $l_1$ norm is used as a differentiable proxy  for the so-called $l_0$ norm of the weights, defined as the number of nonzero elements of a vector:

$$\vert\vert x \vert\vert_0 = \sum_{j=1}^d  \delta(x_j \neq 0) $$

For example, recalling the regression example we saw before, L1 regularization would come in handy to sparsify a model of degree 9 (having 10 parameters) to make it a linear model. We can obtain a similar situation in our XOR problem if we add some meaningless features to the data:

In [ ]:
new_column=torch.randn((X.shape[0],3))
X_noisy=torch.cat([X, new_column], axis=1)

In [ ]:
dataset=torch.utils.data.TensorDataset(X_noisy, Y)
trainset_noisy, testset_noisy=torch.utils.data.random_split(dataset, [500, 500])

trainloader_noisy=torch.utils.data.DataLoader(trainset_noisy, batch_size=16, shuffle=True)
testloader_noisy=torch.utils.data.DataLoader(testset_noisy, batch_size=128, shuffle=False)

Now the input layer of the classifier should have shape 5.

In [ ]:
l1_model=Classifier(5)
lam=0.001
optimizer=torch.optim.Adam(l1_model.parameters(), lr=1e-2)

In [ ]:
epochs=10
losses=[]
test_accuracies=[]
train_accuracies=[]
for epoch in range(epochs):
    print("Test accuracy: ", get_accuracy(l1_model, testloader_noisy))
    l1_model.train()
    print("l1 norm: ", sum(p.abs().sum() for p in l1_model.parameters()))
    print("l0 norm: ", sum(torch.sum(p.abs().ge(1e-2)) for p in l1_model.parameters()))
    
    print("Epoch: ", epoch)
    for x, y in iter(trainloader_noisy):
        out=l1_model(x)
        l=loss(out, y)
        l1_norm=sum(p.abs().sum() for p in l1_model.parameters())
        l+=lam*l1_norm
        optimizer.zero_grad()
        l.backward()
        optimizer.step()
        losses.append(l.item())
print("Final accuracy: ", get_accuracy(l1_model, testloader_noisy))
print("Final l1 norm: ", sum(p.abs().sum() for p in l1_model.parameters()))
print("Final l0 norm: ", sum(torch.sum(p.abs().ge(1e-2)) for p in l1_model.parameters()))

In [ ]:
for p in l1_model.parameters():
    print(p)

## Dropout


Dropout acts by removing (i.e. *zeroing-out*) a random subset of the neurons in a given layer for each forward pass.

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/dropout.png?raw=1" width="500"/>

It has one hyperparameter ($p$), which is the fraction of neurons to be dropped out.

During training, each time a layer with backprop produces an output, a fraction $p$ of that output gets discarded (more precisely, the output of each neuron gets discarded w.p. $p$). This helps in such a way that co-dependence between neurons gets *forgotten* by the network. To say it in simple terms, it forces each neuron to be independent from the output of other neurons within the same layer.

Be careful: since dropout has to apply only during training, we must be careful in activating the switch `model.eval()` when testing our network.

In torch, we find Dropout as a module of `torch.nn`.

In [ ]:
class DropoutClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(in_features=2, out_features=16, bias=True)
        self.dropout = torch.nn.Dropout(p=0.2)
        self.layer2 = torch.nn.Linear(in_features=16, out_features=8, bias=True)
        self.layer3 = torch.nn.Linear(in_features=8, out_features=1, bias=True)
        self.activation = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x, show=False):
        x=self.activation(self.layer1(x))
        if show:
            print("Before Dropout")
            print(x)
        x=self.dropout(x)
        if show:
            print("After Dropout")
            print(x)
        x=self.activation(self.layer2(x))
        x=self.sigmoid(self.layer3(x))
        return x

In [ ]:
do_model=DropoutClassifier()
optimizer=torch.optim.Adam(do_model.parameters(), lr=1e-2)

In [ ]:
train(do_model, optimizer, trainloader, testloader)

We can visualize the effect of Dropout in training and evaluation modes. Note that Dropout outputs are always rescaled of a factor $\frac{1}{1-p}$:

In [ ]:
x,y=next(iter(trainloader))

do_model.train()
do_model(x[0], True)

do_model.eval()
do_model(x[0], True)

## Early stopping

Early stopping is yet another example of regularization technique which relies a lot on practical and experimental observations rather than any supporting theory.

It is based upon the concept of **validation**, which is an assessment mode **additional** to *testing*. Actually, what insofar whe have described as testing is technically a validation.
* a validation dataset may be obtained as result of a random splitting of the original training dataset
* a testing dataset should be obtained instead from a model deployed "in the wild" and should consist of data unseen (from both the model and its architect) during the training and designing phase.

In a normal academic setting it's very hard to obtain a proper testing dataset, so usually the meaning of testing and validation get mixed up a little bit.

Anyway, early stopping requires us to assess the model at each epoch to get a proxy for the testing performance(s) (**validation step**). That should gives us an idea of how the model **learns to generalize** (if it ever does...) during training.

The *theoretical trend* ('90 s), which is pretty much absent in modern Deep Learning due to a lot of modern factors, is the following:

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/generalization.jpg?raw=1" width="500"/>

The idea of early stopping comes from the fact that it is not obvious that more training epochs means better performances: there can be a sweet spot in which we are not at the minimum of the training loss but we are at the optimal point in terms of generalization on the validation set. Then it is a good idea to interrupt the training and fix the model weights for evaluation.

There are many possible criteria to decide when to stop exactly: you can find a complete discussion [here](https://page.mi.fu-berlin.de/prechelt/Biblio/stop_tricks1997.pdf). The simplest criteria stop training when the generalization loss (i.e. the validation loss) is stable or increasing for a number of consecutive epochs.

As anticipated, modern Deep Learning practise suggests that early stopping is not always a good idea, since the trend of the previous plot is often not visible. Something like this can happen instead ([Double Descent Phenomenon](https://openai.com/blog/deep-double-descent/)):

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/dd.png?raw=1" width="500"/>


# Normalization

Now, we will move to normalization techniques, which are aimed at transforming features to be on a similar scale to improve the performance and training stability of the model.

We will have a look at [Batch Normalization (BatchNorm)](https://arxiv.org/abs/1502.03167) and [Layer Normalization (LayerNorm)](https://arxiv.org/pdf/1607.06450.pdf). These two techniques follow the same procedure, rescaling data to have mean 0 and variance 1, but work on different axes:

<img src="https://github.com/lorenzobasile/DeepLearning2022/blob/main/images/normalization.png?raw=1" width="500"/>

Choosing between BatchNorm and LayerNorm (or no normalization layer at all) is a hard task, as there is no fixed rule to do it. As a rule of thumb, Computer Vision architectures often employ BatchNorm and NLP ones LayerNorm, but this is highly variable and the situation is changing with the widespread success of Transformer architectures (originally intended for NLP) on different tasks.

### Batch Normalization

In torch, BatchNorm and LayerNorm are implemented as standard layers in the `nn` package:

In [ ]:
class BNClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(in_features=2, out_features=16, bias=True)
        self.bn1 = torch.nn.BatchNorm1d(num_features=16)
        self.layer2 = torch.nn.Linear(in_features=16, out_features=8, bias=True)
        self.bn2 = torch.nn.BatchNorm1d(num_features=8)
        self.layer3 = torch.nn.Linear(in_features=8, out_features=1, bias=True)
        self.activation = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x=self.bn1(self.activation(self.layer1(x)))
        x=self.bn2(self.activation(self.layer2(x)))
        x=self.sigmoid(self.layer3(x))
        return x

Note that `BatchNorm1d` assumes a 2-D or 3-D input. If you are handling images, which are 4-D (batch size, channel, height, width) you can switch to `BatchNorm2d`.

In [ ]:
bn_model=BNClassifier()
optimizer=torch.optim.Adam(bn_model.parameters(), lr=1e-2)

In [ ]:
train(bn_model, optimizer, trainloader, testloader)

### Layer Normalization

LayerNorm works in a very similar way as BatchNorm, and it accepts as input the shape of the slice on which you want to compute mean and std: for an image it would be something like `normalized_shape=[C,H,W]`.

In [ ]:
class LNClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = torch.nn.Linear(in_features=2, out_features=16, bias=True)
        self.ln1 = torch.nn.LayerNorm(normalized_shape=16)
        self.layer2 = torch.nn.Linear(in_features=16, out_features=8, bias=True)
        self.ln2 = torch.nn.LayerNorm(normalized_shape=8)
        self.layer3 = torch.nn.Linear(in_features=8, out_features=1, bias=True)
        self.activation = torch.nn.ReLU()
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x=self.ln1(self.activation(self.layer1(x)))
        x=self.ln2(self.activation(self.layer2(x)))
        x=self.sigmoid(self.layer3(x))
        return x

In [ ]:
ln_model=LNClassifier()
optimizer=torch.optim.Adam(ln_model.parameters(), lr=1e-2)

In [ ]:
train(ln_model, optimizer, trainloader, testloader)